<a href="https://colab.research.google.com/github/YenLinWu/Web_Scraping/blob/master/%E8%98%8B%E6%9E%9C%E6%97%A5%E5%A0%B1%E7%88%AC%E8%9F%B2(with_Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 連結 Google Drive ( if using Google Colab )

In [ ]:
from google.colab import drive, files
drive.mount( '/content/gdrive' )

In [ ]:
#@title 設定工作目錄
import os
print( f'預設工作目錄：{os.getcwd()}' )

# 指定工作資料夾
Folder_Name = 'AppleDaily' #@param {type:"string"}
WorkingDirectory = '/content/gdrive/My Drive/'+ Folder_Name

if not os.path.exists( WorkingDirectory ):
    os.chdir( '/content/gdrive/My Drive/' ) # Change the current working directory to DrivePath.
    os.mkdir( Folder_Name ) # Create the new directory in DrivePath.
    os.chdir( WorkingDirectory )  
else: os.chdir( WorkingDirectory )

print( f'指定後的工作目錄：{os.getcwd()}' )
#retval = os.getcwd()

預設工作目錄：/content
指定後的工作目錄：/content/gdrive/My Drive/AppleDaily


# 載入套件

In [ ]:
import pandas as pd
import time
import requests, bs4  

# [蘋果日報](https://tw.appledaily.com/home/)   

爬取每日 Top 30 新聞的編號、標題及 URL。   
- 網頁路徑: 首頁 &rarr; 日報 &rarr; 熱門  

![蘋果日報首頁](https://raw.githubusercontent.com/YenLinWu/Web_Scraping/master/Apple%20Daily/Image/Apple%20Daily.png)   

![今日TOP30](https://raw.githubusercontent.com/YenLinWu/Web_Scraping/master/Apple%20Daily/Image/TOP30%20News.png) 

In [ ]:
#@title Step 1: 輸入網站 URL
URL = 'https://tw.appledaily.com/daily/hot/' #@param {type:"string"}

In [ ]:
#@title Step 2: 爬取網頁資訊
#@markdown 使用 `requests.get( URL )` 爬取網頁資訊，將回傳該網頁的 HTML 原始碼。

HTML_File = requests.get( URL )

In [ ]:
#@title Step 3: 檢查是否成功爬取網頁內容
#@markdown 使用 `requests.get( URL ).status_code` 檢查網頁爬蟲的狀態。

try:
    if HTML_File.status_code == 200 : 
        print( str(HTML_File.status_code)+' 請求成功！' )
    HTML_File.raise_for_status( )    # 取得請求錯誤的詳細資訊！
except  requests.exceptions.RequestException as ex1 :
    print( '網頁請求錯誤 : ' + str(ex1) )
except  requests.exceptions.HTTPError as ex2 :
    print( '網頁回應錯誤 : ' + str(ex2) )
except  requests.exceptions.Timeout as ex3 :
    print( 'Timeout 錯誤 : ' + str(ex3) )
except  requests.exceptions.ConnectionError as ex4 :
    print( '網路連線錯誤 : ' + str(ex4) )

200 請求成功！


In [ ]:
#@title Step 4: 列印爬取的網頁 HTML 原始碼內容
print( HTML_File.text )

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="zh-TW" class="ie6 ielt8"> <![endif]-->
<!--[if IE 7 ]>    <html lang="zh-TW" class="ie7 ielt8"> <![endif]-->
<!--[if IE 8 ]>    <html lang="zh-TW" class="ie8"> <![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--> <html lang="zh-TW"> <!--<![endif]-->
<head>
    <!-- Google Tag Manager For AppledailyTW_PRD-->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-W8R48TQ');</script>
<!-- End Google Tag Manager -->    <title>日報｜今日熱門｜蘋果新聞網｜蘋果日報</title>
    <meta property="og:title" content="日報｜今日熱門｜蘋果新聞網｜蘋果日報"/>
    <meta property="og:type" content="website"/>
    <meta property="og:image" content="https://img.appledaily.com.tw/appledaily/images/fbshare/daily_600x315.png"/>
    

## Step 5 : 使用 BeautifulSoup 解析 HTML  

利用 BeautifulSoup 模組中的 lxml 方法解析 HTML 文件。

- Step 5-0 (optional) : 安裝 lxml  
- Step 5-1 : 使用 `bs4.BeautifulSoup( )` 來建立 BeautifulSoup 物件  
- Step 5-2 : 建立空的 Data Frame   
- Step 5-3 : 擷取 HTML 中指定的內容且逐條新聞存入於 Data Frame 中   
  * `find( 'label' )` : 尋找 HTML 中第一個 < 'label' > 標籤內容  
  * `find_all( 'label', limit )` : 尋找 HTML 中所有的 < 'label' > 標籤內容( limit 參數可設定尋找的最大數量 )  
  * `get( 'attr' )` : 取得標籤中 'attr' 屬性的內容 


In [ ]:
# Step 5-0 : 安裝 lxml 
!pip install lxml

In [ ]:
# Step 5-1 : 建立 BeautifulSoup 物件
objSoup = bs4.BeautifulSoup( HTML_File.text, 'lxml' )  

In [ ]:
# Step 5-2 : 建立空的 Data Frame( for 存儲爬蟲的資料 )
df_TopNews = pd.DataFrame( columns=[ '編號', '標題', 'URL' ] )

In [ ]:
# Step 5-3 : 擷取 HTML 的內容且逐條新聞存入於 Data Frame 中
'''
擷取 HTML 標籤中的 <ul class="all"> ......... </ul> 中的內容， 
再篩選出當中所有的 <li> 標籤的內容。 
'''
news = objSoup.find( 'ul', 'all' ).find_all( 'li', limit=10 )

for item in news: 
    
    # 新聞編號
    if item.find( 'div', 'aht_title_num atopres' ): 
        num = item.find( 'div', 'aht_title_num atopres' ).text
    else: 
        num = item.find( 'div', 'aht_title_num' ).text   
    
    # 新聞標題
    title = item.find( 'div', 'aht_title' ).text
    
    # 新聞 URL
    url = item.find( 'a' ).get( 'href' )

    # 新聞資訊存入 Data Frame
    df_TopNews = df_TopNews.append( {'編號':num ,'標題':title,'URL':url}, ignore_index=True )

# 顯示爬蟲資料
print( '今日TOP30( %s ) \n' % time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()+8*3600)) )
df_TopNews

今日TOP30( 2020-06-29 23:31:24 ) 



,編號,標題,URL
0,01,連假期間4利空 台股開盤不妙 陶冬：只要QE不停 牛市就未結束,https://tw.appledaily.com/finance/daily/202006...
1,02,領到10萬紓困就債協 銀行憂倒帳率1成,https://tw.appledaily.com/finance/daily/202006...
2,03,4連勝發大財 中信雙贏 端午假期長紅,https://tw.appledaily.com/sports/daily/2020062...
3,04,繼父潑熱水 4歲童腳潰爛 一度謊稱：她自己弄的 致5%二度燒燙傷 還拖延就醫,https://tw.appledaily.com/headline/daily/20200...
4,05,繼父不當管教 早在3月被列管,https://tw.appledaily.com/headline/daily/20200...
5,06,一字馬鎮全場 范乙霏超軟Q,https://tw.appledaily.com/sports/daily/2020062...
6,07,機師偷吃酒女 被控射後不理 女方痔瘡脫肛 法官：難證性侵,https://tw.appledaily.com/headline/daily/20200...
7,08,王力宏頂學士帽唱畢業歌 祕密基地被點1.4億次,https://tw.appledaily.com/entertainment/daily/...
8,09,台股防守3均線 IC設計人氣不能散,https://tw.appledaily.com/finance/daily/202006...
9,10,周杰倫 炫妻小真身陪聊 IG圈粉600萬霸台星,https://tw.appledaily.com/entertainment/daily/...


In [ ]:
#@title 點閱新聞連結
Item_NO =  1#@param {type:"number"}

if Item_NO <= df_TopNews.shape[0]:
    print( '編號: ', df_TopNews['編號'][Item_NO-1] )
    print( '標題: ', df_TopNews['標題'][Item_NO-1] )
    print( 'URL: ', df_TopNews.URL[Item_NO-1] )
else: 
    print( '爬取 %d 條 TOP30 新聞，請重新輸入 Item_NO 。' % df_TopNews.shape[0] )

編號:  01
標題:  連假期間4利空 台股開盤不妙 陶冬：只要QE不停 牛市就未結束
URL:  https://tw.appledaily.com/finance/daily/20200629/38669691


## Step 6: 將資料寫入 csv 檔   

使用 `to_csv( File_Path )` 將爬到的資料寫入 csv 檔，其中 File_Path 為指定的檔案名稱( 包含其所屬資料夾路徑及副檔名 )。

In [ ]:
File_Name =  "2020_06_29"#@param {type:"string"}
df_TopNews.to_csv( os.getcwd()+'/'+File_Name+'.csv', encoding='utf_8_sig', index=False )

## Step 7: 將檔案從 Google Drive 下載到本機端 

In [ ]:
files.download( File_Name+'.csv' ) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>